## Outlier

In [2]:
import pandas as pd
import numpy as np

default_dir = '/home/users/s17011720/predicting_impression_using_mobile_game/'

In [3]:
asset = pd.read_csv(default_dir + "extract_features_data/asset_day01.csv")
activity = pd.read_csv(default_dir + "extract_features_data/activity_day01.csv")
ads = pd.read_csv(default_dir + "extract_features_data/ads_day01.csv")

# y_df =_csv(default_dir + "extract_features_data/target_y.csv")

In [4]:
def select_target(df):
    target = pd.read_csv(default_dir + "column_filtered_data/target.csv")
    target = target[target['not_target']==0]
    target_id = target['ad_id'].tolist()
    df = df[df['ad_id'].isin(target_id)]
    return df

def select_train(df, YN):
    target = pd.read_csv(default_dir + "column_filtered_data/target.csv")
    target = target[target['not_target']==0 & (df['train_YN']==1)]
    target_id = target['ad_id'].tolist()
    df = df[df['ad_id'].isin(target_id)]
    return df

In [25]:
def outlier_user(df, id_list):
    target = pd.read_csv(default_dir + "column_filtered_data/target.csv")
    
    target['{}_outlier'.format(df)] = 0
    target[target['ad_id'].isin(id_list)]['{}_outlier'.format(df)] = 1
    
    target.to_csv(default_dir + "column_filtered_data/target_add_{}_outlier.csv".format(df), index=False)
    return

In [23]:
def outlier_iqr(data, column): 
    print('--------- '+column+'--------- ')
    # lower, upper 글로벌 변수 선언하기     
#     global lower, upper    
    
    # 4분위수 기준 지정하기     
    q25 = np.quantile(data[column], 0.25) 
    q75 = np.quantile(data[column], 0.75)          
    
    # IQR 계산하기     
    iqr = q75 - q25    
    # outlier cutoff 계산하기     
    cut_off = iqr * 1.5          
    
    # lower와 upper bound 값 구하기     
    lower = q25 - cut_off  
    upper = q75 + cut_off 
    
    print('IQR : ',iqr)     
    print('lower bound : ', lower)     
    print('upper bound : ', upper)    
    
    # 1사 분위와 4사 분위에 속해있는 데이터 각각 저장하기     
    data1 = data[data[column] > upper]     
    data2 = data[data[column] < lower]    
    
    print('총 이상치 개수 : ', data1.shape[0] + data2.shape[0])
    print('해당 데이터에서 이상치의 비율 : {}%'.format(round((data1.shape[0] + data2.shape[0])/data.shape[0]*100,2)))
    
    # 이상치 데이터 id 입력 
    outlier_id = [i for i in (data1.ad_id + data2.ad_id)]
    outlier_user(data, outlier_id)
    
    return data[~data['ad_id'].isin(outlier_id)]

In [19]:
def del_outlier(df):
    
    for col in df.drop('ad_id', axis=1).columns:
        df = outlier_iqr(df, col)
    
    return df

In [12]:
asset.fillna(0, inplace=True)
activity.fillna(0, inplace=True)
ads.fillna(0, inplace=True)

In [24]:
asset_df = del_outlier(asset)

--------- in_coin_cnt_0--------- 
IQR :  10.0
lower bound :  -11.0
upper bound :  29.0
총 이상치 개수 :  15454
해당 데이터에서 이상치의 비율 : 6.88%
--------- out_coin_cnt_0--------- 
IQR :  7.0
lower bound :  -10.5
upper bound :  17.5
총 이상치 개수 :  17647
해당 데이터에서 이상치의 비율 : 7.86%
--------- in_coin_sum_0--------- 
IQR :  1086.0
lower bound :  -1425.0
upper bound :  2919.0
총 이상치 개수 :  20871
해당 데이터에서 이상치의 비율 : 9.3%
--------- out_coin_sum_0--------- 
IQR :  693.0
lower bound :  -1039.5
upper bound :  1732.5
총 이상치 개수 :  27449
해당 데이터에서 이상치의 비율 : 12.23%
--------- curr_var_min_0--------- 
IQR :  27.0
lower bound :  -16.5
upper bound :  91.5
총 이상치 개수 :  1268
해당 데이터에서 이상치의 비율 : 0.56%
--------- curr_var_max_0--------- 
IQR :  401.0
lower bound :  -397.5
upper bound :  1206.5
총 이상치 개수 :  23305
해당 데이터에서 이상치의 비율 : 10.38%
--------- buy_skin_var_sum_0--------- 
IQR :  0.0
lower bound :  0.0
upper bound :  0.0
총 이상치 개수 :  12175
해당 데이터에서 이상치의 비율 : 5.42%
--------- offline_coin_var_sum_0--------- 
IQR :  0.0
lower bound :  0.

(224493, 53)

In [14]:
activity_df = del_outlier(activity)
ads_df = del_outlier(ads)

(224493, 53)